In [18]:
from production import Problem, local_move, ExactSolver, Worldline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [19]:
import matplotlib.patches as patches
from matplotlib.collections import LineCollection

# --- [Assuming Problem, Worldline, and metropolis_move are defined as before] ---
# (If running this in a new session, re-paste the classes from the previous step)

def draw_worldline(wl, ax=None):
    """
    Draws the worldline configuration matching Fig 10.1 style.
    
    Visual Elements:
    - Shaded squares: Interaction plaquettes (checkerboard).
    - Bold lines: Evolution of Up (+1) spins.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
        
    L = wl.problem.n_sites
    M = 2 * wl.problem.m
    
    ax.clear()
    ax.set_xlim(-0.5, L - 0.5)
    ax.set_ylim(0, M)
    ax.set_xlabel("Lattice Sites")
    ax.set_ylabel("Imaginary Time")
    ax.set_title(f"Worldline Loop Algorithm (Sz={int(np.sum(wl.spins)/2)})")
    
    # 1. Draw Checkerboard Background (The Interaction Plaquettes)
    # -----------------------------------------------------------
    # We iterate through time slices.
    # Even t: Bonds (0,1), (2,3)...
    # Odd t:  Bonds (1,2), (3,4)...
    
    for t in range(M):
        # Determine offset for this time slice
        # If t is even, offset is 0. If t is odd, offset is 1.
        offset = t % 2
        
        for i in range(offset, L, 2):
            # The bond connects i and (i+1). 
            # We draw a shaded rectangle for the active plaquette.
            
            # Handle Periodic Boundary Condition for drawing:
            # If i is L-1 (last site) and offset is 1, it connects to 0.
            if i == L - 1:
                # Draw half-box on the right and half-box on the left
                rect_right = patches.Rectangle((i - 0.5, t), 1.0, 1, 
                                             linewidth=0, edgecolor='none', facecolor='#e0e0e0')
                rect_left = patches.Rectangle((-0.5, t), 0.5, 1, 
                                            linewidth=0, edgecolor='none', facecolor='#e0e0e0')
                ax.add_patch(rect_right)
                ax.add_patch(rect_left)
            else:
                # Standard internal box
                # Center of box is between i and i+1. Width covers both sites.
                # Sites are at integer coordinates. Box should span i-0.5 to i+1.5?
                # Actually, in Fig 10.1, the sites are on the edges of the boxes.
                # Let's draw the rectangle spanning from i to i+1.
                rect = patches.Rectangle((i, t), 1, 1, 
                                       linewidth=0, edgecolor='none', facecolor='#cccccc')
                ax.add_patch(rect)

    # 2. Draw Worldlines (The "Up" Spins)
    # -----------------------------------
    # We create a list of line segments.
    segments = []
    
    for t in range(M):
        t_next = (t + 1) % M
        
        # Determine the pairing for this row
        # We only need to check active pairs to see if they cross or go straight
        offset = t % 2
        
        # Iterate over ACTIVE bonds
        for i in range(offset, L, 2):
            j = (i + 1) % L
            
            # Get spins at current time t
            s_i = wl.spins[t, i]
            s_j = wl.spins[t, j]
            
            # Get spins at next time t+1
            s_i_next = wl.spins[t_next, i]
            s_j_next = wl.spins[t_next, j]
            
            # --- Logic for Site i ---
            if s_i == 1: # If spin is UP, we draw a line
                start_point = (i, t)
                
                # Where did it go?
                # Since particle number is conserved in the plaquette:
                # If s_i_next is 1, it went straight (usually).
                # If s_j_next is 1 (and s_i_next is -1), it swapped.
                
                # Note: This logic assumes conservation. If MC temporarily breaks it, visual might look weird.
                if s_i_next == 1:
                    # Straight up
                    end_point = (i, t + 1)
                    segments.append([start_point, end_point])
                elif s_j_next == 1:
                    # Diagonal swap to j
                    # Handle visual wrapping for PBC
                    if i == L-1 and j == 0:
                        # Don't draw a line across screen. Draw two partials?
                        # For simplicity, we skip drawing the wrap-around line to avoid clutter
                        pass 
                    else:
                        end_point = (j, t + 1)
                        segments.append([start_point, end_point])

            # --- Logic for Site j ---
            if s_j == 1:
                start_point = (j, t)
                if s_j_next == 1:
                    # Straight up
                    end_point = (j, t + 1)
                    segments.append([start_point, end_point])
                elif s_i_next == 1:
                    # Diagonal swap to i
                    if i == L-1 and j == 0:
                        pass
                    else:
                        end_point = (i, t + 1)
                        segments.append([start_point, end_point])

    # Add lines to plot
    lc = LineCollection(segments, colors='black', linewidths=2.5)
    ax.add_collection(lc)
    
    # Add vertical grid lines for reference (sites)
    ax.set_xticks(range(L))
    ax.grid(axis='x', linestyle=':', alpha=0.3)

In [28]:
def animate_worldlines(pb):
    # Setup problem
    wl = Worldline(pb)
    
    # Force a "striped" start (Up, Down, Up, Down) for visual clarity
    wl.spins = np.ones((2*pb.m, pb.n_sites))
    for r in range(2*pb.m):
        for c in range(pb.n_sites):
            if c % 2 == 1:
                wl.spins[r,c] = -1
            
    fig, ax = plt.subplots(figsize=(10, 6))

    print("Burning phase...", end="")
    for _ in range(5000):
        local_move(wl)
    print("done")
    
    energies = []
    weights = []

    def update(frame):
        # Do a few MC steps
        for _ in range(pb.n_sites * 2 * pb.m):
            local_move(wl)

        energies.append(wl.energy)
        weights.append(wl.weight)
            
        draw_worldline(wl, ax)
        return ax.artists + ax.collections


    # Create animation
    ani = FuncAnimation(fig, update, frames=500, interval=200, blit=False)
    
    # Save to gif for portability
    ani.save("worldline_checkerboard.gif", writer='pillow', fps=10)
    print(f"Got {len(energies)} values for ernergies.")
    print(f"mean: {np.mean(energies)}, unnormalized weighted mean: {np.mean(np.array(energies) * np.array(weights))}, normalized weighted mean: {np.mean(np.array(energies) * np.array(weights)) / np.sum(weights)}")
    print("Animation saved to worldline_checkerboard.gif")

In [31]:
pb = Problem(n_sites=6, J_x=1.0, J_z=1.0, temperature=100.0, m=4)
solver = ExactSolver(pb)
print(solver.energy)

(-0.007506083110480158-2.8190051742400254e-23j)


In [32]:
animate_worldlines(pb)

Burning phase...done


: 